# Jupyter notebook sample

In [1]:
# Install ffmpeg using conda
!conda install -c conda-forge ffmpeg -y

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.10.3
  latest version: 24.9.2

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /Users/yuiwatanabe/opt/anaconda3/envs/telegram_bot

  added / updated specs:
    - ffmpeg


The following NEW packages will be INSTALLED:

  aom                pkgs/main/osx-64::aom-3.6.0-hcec6c5f_0
  cairo              pkgs/main/osx-64::cairo-1.16.0-h3ce6f7e_5
  dav1d              conda-forge/osx-64::dav1d-1.2.1-h0dc2134_0
  expat              conda-forge/osx-64::expat-2.6.3-hac325c4_0
  ffmpeg             pkgs/main/osx-64::ffmpeg-6.1.1-h931e7ea_0
  fontconfig         pkgs/main/osx-64::fontconfig-2.14.1-hb0a0c50_2
  fribidi            conda-forge/osx-64::fribidi-1.0.10-hbcb3906_0
  gdk-pixbuf         pkgs/main/osx-64::gdk-pixbuf-2.42.10-h46256e1_1
  giflib             conda-forge/osx-64::giflib-5.2.2-h10d778d_0
  graphite2          pkg

In [6]:
!pip install git+https://github.com/openai/whisper.git
# Install ffmpeg without sudo
!apt update && apt install -y ffmpeg

  Cloning https://github.com/openai/whisper.git to /private/var/folders/y9/ynp6f9z95sv1f6g_g09bwmkr0000gn/T/pip-req-build-d7skx3jv
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /private/var/folders/y9/ynp6f9z95sv1f6g_g09bwmkr0000gn/T/pip-req-build-d7skx3jv
  Resolved https://github.com/openai/whisper.git to commit 25639fc17ddc013d56c594bfbf7644f2185fad84
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached numba-0.58.1-cp38-cp38-macosx_10_9_x86_64.whl.metadata (2.7 kB)
  Using cached torch-2.2.2-cp38-none-macosx_10_9_x86_64.whl.metadata (25 kB)
  Using cached tqdm-4.66.5-py3-none-any.whl.metadata (57 kB)
  Using cached more_itertools-10.5.0-py3-none-any.whl.metadata (36 kB)
  Using cached tiktoken-0.7.0-cp38-cp38-macosx_10_9_x86_64.whl.metadata (6.6 kB)
  Using cached llvmlite-0.41.1-cp38-cp38-macosx_10_9_x86_64.whl.metadata (4.8 kB)
  Using ca

### Load audio

In [7]:
import subprocess

def transcribe_audio(ogg_file):
    # Generate the .txt file using whisper
    txt_file = ogg_file.replace('.ogg', '.txt')
    subprocess.run(['whisper', ogg_file, '--model', 'tiny', '--language', 'en'], check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

    # Read and print the content of the generated .txt file
    with open(txt_file, 'r') as file:
        content = file.read()
        print(content)
        

In [2]:
import requests
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

TELEGRAM_TOKEN = os.getenv("TELEGRAM_TOKEN")
base_url = f"https://api.telegram.org/bot{TELEGRAM_TOKEN}"

# Step 1: Get updates and extract file_id for voice messages
get_updates_url = f"{base_url}/getUpdates"
parameters = {"offset": "100"}
resp = requests.get(get_updates_url, params=parameters)
updates = resp.json()

# Iterate through updates to find voice messages
cnt = 0
for update in updates['result']:
    if 'voice' in update['message']:
        file_id = update['message']['voice']['file_id']
        print(f"file_id: {file_id}")

        # Step 2: Get file path
        get_file_url = f"{base_url}/getFile"
        file_params = {"file_id": file_id}
        file_resp = requests.get(get_file_url, params=file_params)
        file_path = file_resp.json()['result']['file_path']

        # Step 3: Download the file
        file_url = f"https://api.telegram.org/file/bot{TELEGRAM_TOKEN}/{file_path}"
        voice_file = requests.get(file_url)

        # Save the file
        output = 'voice_' + str(cnt) + '.ogg'
        with open(output, 'wb') as f:
            f.write(voice_file.content)

        cnt += 1
        print("Voice message downloaded successfully.")

file_id: AwACAgQAAxkBAAMFZxO_Pc2cZRDsiDyx8ESws5Z3QjIAAgQfAAKpZaFQZaMwyS0j4442BA
Voice message downloaded successfully.
file_id: AwACAgQAAxkBAAMIZxPOa3twVEYdSuQv7_BeFmfDSS8AAlQfAAKpZaFQO3JCXBrogQw2BA
Voice message downloaded successfully.
file_id: AwACAgQAAxkBAAMJZxPOcHD9l3ubhevppFv26DREV-MAAlUfAAKpZaFQ5ISIDEYf0Xw2BA
Voice message downloaded successfully.


In [8]:
import subprocess
import requests
import os

load_dotenv()

TELEGRAM_TOKEN = os.getenv("TELEGRAM_TOKEN")
base_url = f"https://api.telegram.org/bot{TELEGRAM_TOKEN}"

# Step 1: Get updates
get_updates_url = f"{base_url}/getUpdates"
parameters = {"offset": "100"}
resp = requests.get(get_updates_url, params=parameters)
updates = resp.json()

# Step 2: Count voice messages
voice_count = 0
for update in updates['result']:
    if 'voice' in update['message']:
        voice_count += 1

print(f"Number of voice messages: {voice_count}")

# Transcribe all voice_cnt.ogg files
for cnt in range(voice_count):  # Replace voice_count with the actual number of voice files
    file_name = f'voice_{cnt}.ogg'
    if os.path.exists(file_name):
        transcribe_audio(file_name)
    else:
        print(f"File {file_name} does not exist.")

Number of voice messages: 3
Hello

Hello, too

Hello, it's me.

